In [1]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as f
from pyspark.sql.functions import col, signum

spark = SparkSession.builder.master("local[1]").getOrCreate()
df=spark.read.parquet("CleanedSpy.parquet")

In [2]:
#Pred pandas df for timeseriescv
pdDf=df.toPandas()
pdDf=pdDf.rename(columns={"Time": "pred_times"})
pdDf['eval_times']=pdDf['pred_times']+60;
display(pdDf)

,Ticker,pred_times,Close,Volume,label,features,eval_times
0,1,1562611080,296.7300,48808.0,0.000067,"[1.0, 296.73, 48808.0]",1562611140
1,1,1562612340,296.7100,91917.0,0.000337,"[1.0, 296.71, 91917.0]",1562612400
2,1,1562698140,296.6969,88602.0,-0.000023,"[1.0, 296.6969, 88602.0]",1562698200
3,1,1562771880,296.5000,10000.0,-0.000270,"[1.0, 296.5, 10000.0]",1562771940
4,1,1562800800,298.4100,40558.0,0.000005,"[1.0, 298.41, 40558.0]",1562800860
...,...,...,...,...,...,...,...
118178,1,1592847780,307.4000,253004.0,0.000065,"[1.0, 307.4, 253004.0]",1592847840
118179,1,1592857800,310.1900,53208.0,-0.000032,"[1.0, 310.19, 53208.0]",1592857860
118180,1,1592859660,310.5900,69776.0,0.000451,"[1.0, 310.59, 69776.0]",1592859720
118181,1,1592862540,310.5654,57509.0,-0.000372,"[1.0, 310.5654, 57509.0]",1592862600


In [3]:
#USING MLFLOW library instead
from timeseriescv import cross_validation as cv
crossval=cv.PurgedWalkForwardCV(n_splits=4) #Default settings: n_splits=10, n_test_splits=1, min_train_splits=2, max_train_splits=None
crossval=crossval.split(pdDf, pdDf['label'],pdDf['pred_times'],pdDf['eval_times'],True)

In [4]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8],
    'n_estimators': [100]
}

feat_rows=df.select("features").collect()
feat_list=[row.features for row in feat_rows]
label_rows=df.select("label").collect()
label_list=[row.label for row in label_rows]



In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = crossval, n_jobs = -1, verbose = 2)
#Add scoring for grid search (neg_log_loss?)
grid_search.fit(feat_list, label_list)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:   17.6s remaining:   35.3s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  1.9min finished


In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    return accuracy

In [ ]:
print(grid_search.best_params_)